In [1]:
import Class_diff_hopping_hdf5 as hop
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
# path = r"C:/Users/vlagerweij/Documents/TU jaar 6/Project KOH(aq)/Repros/RPBE_Production/MLMD/100ps_Exp_Density/i_1/"
path = r"C:/Users/vlagerweij/Documents/TU jaar 6/Project KOH(aq)/Repros/RPBE_Production/AIMD/10ps/i_1/"

Traj = hop.Prot_Hop(path, dt=1)
index, loc_OH, loc_K, loc_H2O = Traj.loading()
n_OH = Traj.N_OH

loading file with path =  C:/Users/vlagerweij/Documents/TU jaar 6/Project KOH(aq)/Repros/RPBE_Production/AIMD/10ps/i_1//traj.npz


In [3]:
path = r"C:/Users/vlagerweij\Documents/TU jaar 6/Project KOH(aq)/Progress_meeting_12/Sana new Results/margin_31/"
n_steps = 10000  # ten-thousend timesteps
data_OH = pd.read_csv(path+'trajectory_unwrapped_OH_stats.txt', delim_whitespace=True, header=None).to_numpy()
data_HB = pd.read_csv(path+'trajectory_unwrapped_HB_type_stats.txt', delim_whitespace=True, header=None).to_numpy()
OH_i = np.ones(n_steps)*Traj.N_H
n_HB = np.zeros(n_steps)
n_OHs = np.zeros(n_steps)

In [5]:
limit1 = 4000
limit2 = data_OH.shape[0]-1
limit3 = data_HB.shape[0]-1
# Loop over all timesteps
for i in range(limit1):
    lines_i = np.where((data_OH[:limit2, 0] == i) & (data_OH[:limit2, 3] == 1))
    if len(lines_i[0]) ==1 :
        OH_i[i] = data_OH[lines_i, 2]
    else:
        OH_i[i] = OH_i[i-1]
    
    lines_i = np.where((data_HB[:limit3, 0] == i) & (data_HB[:limit3, 3] == 'wp'))
    if i == 2980:
        print(lines_i[0] + 1)
        print(data_HB[lines_i, 0])
    n_HB[i] = len(data_HB[lines_i, 0])

C:\Users\vlagerweij\AppData\Local\Temp\ipykernel_4400\2692539098.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  OH_i[i] = data_OH[lines_i, 2]


[166904 166922]
[[134 152]]


ValueError: setting an array element with a sequence.

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

fig.add_trace(go.Scatter(x=Traj.t, y=index, name="My method"), row=1, col=1)
fig.add_trace(go.Scatter(x=Traj.t, y=OH_i, name="Sana"), row=1, col=1)

fig.add_trace(go.Scatter(x=Traj.t, y=n_HB, name="Sana"), row=2, col=1)

fig.update_yaxes(title_text="Index of oxygen atom", row=1, col=1)
fig.update_yaxes(title_text="Number of Hydrogen Bonds", row=2, col=1)
fig.update_xaxes(title_text="timestep", row=2, col=1)
fig.update_layout(height=700, width=1100, title_text="Differences between methods CovrH = 0.31")
fig.show()

In [ ]:
import numpy as np

L = 2
n = 1
OH_i_new = np.array([1, 2, 5, 0])
pos_O = np.array([[[0, 0, 0], [1, 1, 1], [1, 1.5, 0], [0.5, 0.5, 0], [0.5, 1, 1], [1, 1, 0]], [[0.1, 0.1, 0.1], [0.9, 0.9, 0.9], [0.9, 0, 0], [0.6, 0.6, 0], [0.4, 1, 1], [1.1, 1.1, 0]]])
OH_i = np.array([[1, 2, 4], [0, 0, 0]])

diff_new = np.setdiff1d(OH_i_new, OH_i[n-1, :], assume_unique=True)
diff_old = np.setdiff1d(OH_i[n-1, :], OH_i_new, assume_unique=True)

In [ ]:
print(diff_new)
print(diff_old)

In [ ]:
OH_i[n, :] = OH_i[n-1, :]
for i in range(len(diff_old)):
    ## HYDROXIDE PART
    # Check every closest old version to every unmatching new one and replace the correct OH_i index
    r_OO = (pos_O[n, diff_new, :] - pos_O[n-1, diff_old[i], :] + L/2) % L - L/2         
    d2 = np.sum(r_OO**2, axis=1)
    i_n = np.argmin(d2)
    print(d2, 'and', i_n)
    if d2[i_n] > 9:  # excape when jump too large
        print("Something went wrong, reaction jump too far, d = ", np.sqrt(d2[i_n]), 'Angstrom',
                            "CPU rank=")

    idx_OH_i = np.where(OH_i[n-1, :] == diff_old[i]) # get index in OH_i storage list
    OH_i[n, idx_OH_i] = diff_new[i_n]  # update OH_i storage list

In [ ]:
OH_i 